# Procesamiento de viajes de Ecobici

__Descripción:__

Se obtiene el AGEB de origen y destino de los viajes de los viajes de ecobici y se obtiene el agregado primedio de los viajes entre AGEBS por hora.

__Input__

- Estaciones de ecobici: Estaciones de ecobici con clave AGEB obtenida de __Uber © 2019 Copyright Uber Technologies, Inc.Data Attributions__

- Viajes Ecobici 2019
Responsable: Ecobici



In [1]:
import pandas as pd

## Estaciones

In [2]:
df_estaciones = pd.read_csv('../data/SIG/procesado/CSV/estaciones-de-ecobici_con_AGEB.csv',sep=',')

In [3]:
df_estaciones['CVE_AGEB']  = [''.join(filter(lambda x: not x.isdigit(), row)) for row in df_estaciones['CVE_AGEB']]

In [4]:
lista_agebs_bien = df_estaciones[df_estaciones['CVE_AGEB']!="A"].index

In [5]:
df_estaciones = df_estaciones.iloc[lista_agebs_bien,:]

In [6]:
df_estaciones = pd.read_csv('../data/SIG/procesado/CSV/estaciones-de-ecobici_con_AGEB.csv',sep=',')
df_estaciones = df_estaciones.iloc[lista_agebs_bien,:]
df_estaciones['CVE_AGEB'] = [''.join(filter(lambda x: x.isdigit(), row)) for row in df_estaciones['CVE_AGEB']]
df_estaciones['CVE_AGEB'] = df_estaciones['CVE_AGEB'].astype(int)

df_estaciones.sort_values("ID").head(5)

,ID,Nombre,Dirección,Número,Código po,districtCo,Colonia,altitude,nearbyStat,location/l,...,nearbySt_3,nearbySt_4,nearbySt_5,punto_geo,CVE_AGEB,CVE_MUN,CVE_LOC,CVE_ENT,MOVEMENT_I,DISPLAY_NA
295,1,1 RIO SENA-RIO BALSAS,001 - Río Sena-Río Balsas,S/N,6500.0,CUA,Cuauhtémoc,NaN,3,19.433571,...,NaN,NaN,NaN,"19.4335714,-99.1678091",790,15,1,9,4749,Movement Zone 4749
173,2,2 RIO GUADALQUIVIR-RIO BALSAS,002 - Río Guadalquivir - Río Balsas,S/N,6500.0,CUA,Cuauhtémoc,NaN,1,19.431386,...,NaN,NaN,NaN,"19.431386,-99.171695",926,15,1,9,4661,Movement Zone 4661
288,3,3 REFORMA-INSURGENTES,003 - Reforma - Insurgentes,S/N,6500.0,1,Ampliación Granada,NaN,8,19.431655,...,NaN,NaN,NaN,"19.431655,-99.158668",729,15,1,9,4746,Movement Zone 4746
172,4,4 RIO NILO-RIO PANUCO,004 - Río Nilo - Río Panuco,S/N,6500.0,CUA,Cuauhtémoc,NaN,5,19.428309,...,10.0,NaN,NaN,"19.428309,-99.171713",926,15,1,9,4661,Movement Zone 4661
364,5,5 RIO PANUCO-RIO TIBER,005 - Río Pánuco Río Tiber,S/N,6500.0,CUA,Cuauhtémoc,NaN,2,19.429720,...,10.0,NaN,NaN,"19.42972,-99.169363",822,15,1,9,4800,Movement Zone 4800


## Prueba

## Viajes ecobici

In [7]:
df_viajes = pd.read_csv('../data/ecobici/ecobici_2019.csv',sep=',')
df_viajes.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,1,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Bici,Ciclo_Estacion_Arribo,Ciclo_Estacion_Retiro,Edad_Usuario,Fecha_Arribo,Fecha_Retiro,Genero_Usuario,Hora_Arribo,Hora_Retiro,Unnamed: 9
0,11444,36,67,36,01/01/2019,01/01/2019,M,0:14:03,0:08:44,NaN
1,9196,136,55,36,01/01/2019,01/01/2019,M,0:42:11,0:12:05,NaN
2,2612,115,29,38,01/01/2019,01/01/2019,M,0:37:47,0:13:15,NaN
3,7463,115,29,43,01/01/2019,01/01/2019,M,0:38:04,0:13:36,NaN
4,3987,35,136,37,01/01/2019,01/01/2019,M,0:22:47,0:15:30,NaN


### merge viajes e info de estaciones

Se obtiene ageb por estación de origen y destino.

#### Clave AGEB arribo

In [8]:
df_viajes_ageb = df_viajes.merge(df_estaciones[['ID','CVE_AGEB',"Nombre"]], 
                                 left_on='Ciclo_Estacion_Retiro', right_on='ID', how="left")

In [9]:
df_viajes_ageb.rename(columns={'CVE_AGEB': 'CVE_AGEB_retiro'}, inplace=True)
df_viajes_ageb = df_viajes_ageb.drop("Unnamed: 9",axis=1)

#### Clave AGEB destino

In [10]:
df_viajes_ageb = df_viajes_ageb.merge(df_estaciones[['ID','CVE_AGEB',"Nombre"]],
                                      left_on='Ciclo_Estacion_Arribo', right_on='ID',how="left")

In [11]:
df_viajes_ageb.rename(columns={'CVE_AGEB': 'CVE_AGEB_arribo'}, inplace=True)

In [12]:
df_viajes_ageb.head()

,Bici,Ciclo_Estacion_Arribo,Ciclo_Estacion_Retiro,Edad_Usuario,Fecha_Arribo,Fecha_Retiro,Genero_Usuario,Hora_Arribo,Hora_Retiro,ID_x,CVE_AGEB_retiro,Nombre_x,ID_y,CVE_AGEB_arribo,Nombre_y
0,11444,36,67,36,01/01/2019,01/01/2019,M,0:14:03,0:08:44,67.0,1318.0,67 VICENTE SUAREZ-MICHOACAN,36.0,1089.0,36 PUEBLA-VERACRUZ
1,9196,136,55,36,01/01/2019,01/01/2019,M,0:42:11,0:12:05,55.0,767.0,55 5 DE MAYO-BOLIVAR,136.0,1002.0,136 ÁLVARO OBREGÓN-TONALÁ
2,2612,115,29,38,01/01/2019,01/01/2019,M,0:37:47,0:13:15,29.0,733.0,29 REFORMA-BUCARELI,115.0,803.0,115 ROMA-DINAMARCA
3,7463,115,29,43,01/01/2019,01/01/2019,M,0:38:04,0:13:36,29.0,733.0,29 REFORMA-BUCARELI,115.0,803.0,115 ROMA-DINAMARCA
4,3987,35,136,37,01/01/2019,01/01/2019,M,0:22:47,0:15:30,136.0,1002.0,136 ÁLVARO OBREGÓN-TONALÁ,NaN,NaN,NaN


## Renombramos las columnas de nombre:

In [13]:
df_viajes_ageb.rename({"Nombre_x":"nombre_estacion_retiro","Nombre_y":"nombre_estacion_arribo"},axis=1,inplace=True)
df_viajes_ageb.head()

,Bici,Ciclo_Estacion_Arribo,Ciclo_Estacion_Retiro,Edad_Usuario,Fecha_Arribo,Fecha_Retiro,Genero_Usuario,Hora_Arribo,Hora_Retiro,ID_x,CVE_AGEB_retiro,nombre_estacion_retiro,ID_y,CVE_AGEB_arribo,nombre_estacion_arribo
0,11444,36,67,36,01/01/2019,01/01/2019,M,0:14:03,0:08:44,67.0,1318.0,67 VICENTE SUAREZ-MICHOACAN,36.0,1089.0,36 PUEBLA-VERACRUZ
1,9196,136,55,36,01/01/2019,01/01/2019,M,0:42:11,0:12:05,55.0,767.0,55 5 DE MAYO-BOLIVAR,136.0,1002.0,136 ÁLVARO OBREGÓN-TONALÁ
2,2612,115,29,38,01/01/2019,01/01/2019,M,0:37:47,0:13:15,29.0,733.0,29 REFORMA-BUCARELI,115.0,803.0,115 ROMA-DINAMARCA
3,7463,115,29,43,01/01/2019,01/01/2019,M,0:38:04,0:13:36,29.0,733.0,29 REFORMA-BUCARELI,115.0,803.0,115 ROMA-DINAMARCA
4,3987,35,136,37,01/01/2019,01/01/2019,M,0:22:47,0:15:30,136.0,1002.0,136 ÁLVARO OBREGÓN-TONALÁ,NaN,NaN,NaN


### Eliminamos columnas no usadas

In [14]:
df_viajes_ageb.columns

Index(['Bici', 'Ciclo_Estacion_Arribo', 'Ciclo_Estacion_Retiro',
       'Edad_Usuario', 'Fecha_Arribo', 'Fecha_Retiro', 'Genero_Usuario',
       'Hora_Arribo', 'Hora_Retiro', 'ID_x', 'CVE_AGEB_retiro',
       'nombre_estacion_retiro', 'ID_y', 'CVE_AGEB_arribo',
       'nombre_estacion_arribo'],
      dtype='object')

In [15]:
df_viajes_ageb.drop(['ID_x','ID_y'], axis=1, inplace=True)

In [16]:
df_viajes_ageb.head()

,Bici,Ciclo_Estacion_Arribo,Ciclo_Estacion_Retiro,Edad_Usuario,Fecha_Arribo,Fecha_Retiro,Genero_Usuario,Hora_Arribo,Hora_Retiro,CVE_AGEB_retiro,nombre_estacion_retiro,CVE_AGEB_arribo,nombre_estacion_arribo
0,11444,36,67,36,01/01/2019,01/01/2019,M,0:14:03,0:08:44,1318.0,67 VICENTE SUAREZ-MICHOACAN,1089.0,36 PUEBLA-VERACRUZ
1,9196,136,55,36,01/01/2019,01/01/2019,M,0:42:11,0:12:05,767.0,55 5 DE MAYO-BOLIVAR,1002.0,136 ÁLVARO OBREGÓN-TONALÁ
2,2612,115,29,38,01/01/2019,01/01/2019,M,0:37:47,0:13:15,733.0,29 REFORMA-BUCARELI,803.0,115 ROMA-DINAMARCA
3,7463,115,29,43,01/01/2019,01/01/2019,M,0:38:04,0:13:36,733.0,29 REFORMA-BUCARELI,803.0,115 ROMA-DINAMARCA
4,3987,35,136,37,01/01/2019,01/01/2019,M,0:22:47,0:15:30,1002.0,136 ÁLVARO OBREGÓN-TONALÁ,NaN,NaN


## Obtenemos la columna de hora de retiro:

In [17]:
df_viajes_ageb['Hora_Retiro_round'] = df_viajes_ageb['Hora_Retiro'].str.split(':').str[0].astype(int)

In [18]:
df_viajes_ageb.head()

,Bici,Ciclo_Estacion_Arribo,Ciclo_Estacion_Retiro,Edad_Usuario,Fecha_Arribo,Fecha_Retiro,Genero_Usuario,Hora_Arribo,Hora_Retiro,CVE_AGEB_retiro,nombre_estacion_retiro,CVE_AGEB_arribo,nombre_estacion_arribo,Hora_Retiro_round
0,11444,36,67,36,01/01/2019,01/01/2019,M,0:14:03,0:08:44,1318.0,67 VICENTE SUAREZ-MICHOACAN,1089.0,36 PUEBLA-VERACRUZ,0
1,9196,136,55,36,01/01/2019,01/01/2019,M,0:42:11,0:12:05,767.0,55 5 DE MAYO-BOLIVAR,1002.0,136 ÁLVARO OBREGÓN-TONALÁ,0
2,2612,115,29,38,01/01/2019,01/01/2019,M,0:37:47,0:13:15,733.0,29 REFORMA-BUCARELI,803.0,115 ROMA-DINAMARCA,0
3,7463,115,29,43,01/01/2019,01/01/2019,M,0:38:04,0:13:36,733.0,29 REFORMA-BUCARELI,803.0,115 ROMA-DINAMARCA,0
4,3987,35,136,37,01/01/2019,01/01/2019,M,0:22:47,0:15:30,1002.0,136 ÁLVARO OBREGÓN-TONALÁ,NaN,NaN,0


## Obtenemos el tiempo de cada viaje:

Eliminamos filas con errores de formato:

In [19]:
df_viajes_ageb[df_viajes_ageb['Hora_Retiro']=='18::']

,Bici,Ciclo_Estacion_Arribo,Ciclo_Estacion_Retiro,Edad_Usuario,Fecha_Arribo,Fecha_Retiro,Genero_Usuario,Hora_Arribo,Hora_Retiro,CVE_AGEB_retiro,nombre_estacion_retiro,CVE_AGEB_arribo,nombre_estacion_arribo,Hora_Retiro_round
2758143,8026,400,18,19,28/04/2019,26/04/2019,M,01:15:10,18::,837.0,18 REFORMA-RIO RHIN,755.0,400 SAN LORENZO-HERIBERTO FRIAS,18


In [20]:
index_number = df_viajes_ageb[df_viajes_ageb['Hora_Retiro']=='18::'].index.values[0]
df_viajes_ageb.drop([index_number],inplace=True)

Convetimos a datetime:

In [21]:
df_viajes_ageb['Hora_Retiro'] = pd.to_datetime(df_viajes_ageb['Hora_Retiro'], format='%H:%M:%S')
df_viajes_ageb['Hora_Arribo'] = pd.to_datetime(df_viajes_ageb['Hora_Arribo'], format='%H:%M:%S')

Verificamos que las horas de arribo y reutro sean datetime

restamos las columnas para obtener el tiempo

In [22]:
df_viajes_ageb['duracion_viaje'] = df_viajes_ageb['Hora_Arribo'] - df_viajes_ageb['Hora_Retiro']

In [23]:
df_viajes_ageb['duracion_viaje_minutos'] = df_viajes_ageb['duracion_viaje'].astype('str').str.split(':').str[1].astype(int)

In [24]:
df_viajes_ageb.head()

,Bici,Ciclo_Estacion_Arribo,Ciclo_Estacion_Retiro,Edad_Usuario,Fecha_Arribo,Fecha_Retiro,Genero_Usuario,Hora_Arribo,Hora_Retiro,CVE_AGEB_retiro,nombre_estacion_retiro,CVE_AGEB_arribo,nombre_estacion_arribo,Hora_Retiro_round,duracion_viaje,duracion_viaje_minutos
0,11444,36,67,36,01/01/2019,01/01/2019,M,1900-01-01 00:14:03,1900-01-01 00:08:44,1318.0,67 VICENTE SUAREZ-MICHOACAN,1089.0,36 PUEBLA-VERACRUZ,0,00:05:19,5
1,9196,136,55,36,01/01/2019,01/01/2019,M,1900-01-01 00:42:11,1900-01-01 00:12:05,767.0,55 5 DE MAYO-BOLIVAR,1002.0,136 ÁLVARO OBREGÓN-TONALÁ,0,00:30:06,30
2,2612,115,29,38,01/01/2019,01/01/2019,M,1900-01-01 00:37:47,1900-01-01 00:13:15,733.0,29 REFORMA-BUCARELI,803.0,115 ROMA-DINAMARCA,0,00:24:32,24
3,7463,115,29,43,01/01/2019,01/01/2019,M,1900-01-01 00:38:04,1900-01-01 00:13:36,733.0,29 REFORMA-BUCARELI,803.0,115 ROMA-DINAMARCA,0,00:24:28,24
4,3987,35,136,37,01/01/2019,01/01/2019,M,1900-01-01 00:22:47,1900-01-01 00:15:30,1002.0,136 ÁLVARO OBREGÓN-TONALÁ,NaN,NaN,0,00:07:17,7


In [25]:
df_viajes_ageb.drop(['Bici','Fecha_Arribo',
                    'Fecha_Retiro','duracion_viaje','Hora_Arribo','Hora_Retiro'], axis=1, inplace=True)

In [26]:
df_viajes_ageb.head()

,Ciclo_Estacion_Arribo,Ciclo_Estacion_Retiro,Edad_Usuario,Genero_Usuario,CVE_AGEB_retiro,nombre_estacion_retiro,CVE_AGEB_arribo,nombre_estacion_arribo,Hora_Retiro_round,duracion_viaje_minutos
0,36,67,36,M,1318.0,67 VICENTE SUAREZ-MICHOACAN,1089.0,36 PUEBLA-VERACRUZ,0,5
1,136,55,36,M,767.0,55 5 DE MAYO-BOLIVAR,1002.0,136 ÁLVARO OBREGÓN-TONALÁ,0,30
2,115,29,38,M,733.0,29 REFORMA-BUCARELI,803.0,115 ROMA-DINAMARCA,0,24
3,115,29,43,M,733.0,29 REFORMA-BUCARELI,803.0,115 ROMA-DINAMARCA,0,24
4,35,136,37,M,1002.0,136 ÁLVARO OBREGÓN-TONALÁ,NaN,NaN,0,7


### Estos datos se usaran para obtener los viajes de una estación a otra por ageb.

In [27]:
df_viajes_ageb.to_csv('../data/procesado/datos_ecobici_por_ageb.csv',index=False)

In [28]:
df_viajes_ageb.head()

,Ciclo_Estacion_Arribo,Ciclo_Estacion_Retiro,Edad_Usuario,Genero_Usuario,CVE_AGEB_retiro,nombre_estacion_retiro,CVE_AGEB_arribo,nombre_estacion_arribo,Hora_Retiro_round,duracion_viaje_minutos
0,36,67,36,M,1318.0,67 VICENTE SUAREZ-MICHOACAN,1089.0,36 PUEBLA-VERACRUZ,0,5
1,136,55,36,M,767.0,55 5 DE MAYO-BOLIVAR,1002.0,136 ÁLVARO OBREGÓN-TONALÁ,0,30
2,115,29,38,M,733.0,29 REFORMA-BUCARELI,803.0,115 ROMA-DINAMARCA,0,24
3,115,29,43,M,733.0,29 REFORMA-BUCARELI,803.0,115 ROMA-DINAMARCA,0,24
4,35,136,37,M,1002.0,136 ÁLVARO OBREGÓN-TONALÁ,NaN,NaN,0,7


### Obtención de genero de usuarios por viaje:

In [29]:
df_viajes_ageb = pd.read_csv('../data/procesado/datos_ecobici_por_ageb.csv')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
df_viajes_ageb['Genero_Usuario_F'] = pd.get_dummies(df_viajes_ageb['Genero_Usuario'])["F"]
df_viajes_ageb['Genero_Usuario_M'] = pd.get_dummies(df_viajes_ageb['Genero_Usuario'])["M"]

In [31]:
print(df_viajes_ageb['CVE_AGEB_retiro'].nunique(),df_viajes_ageb['CVE_AGEB_arribo'].nunique())

128 128


In [32]:
df_viajes_ageb.head()

,Ciclo_Estacion_Arribo,Ciclo_Estacion_Retiro,Edad_Usuario,Genero_Usuario,CVE_AGEB_retiro,nombre_estacion_retiro,CVE_AGEB_arribo,nombre_estacion_arribo,Hora_Retiro_round,duracion_viaje_minutos,Genero_Usuario_F,Genero_Usuario_M
0,36,67,36,M,1318.0,67 VICENTE SUAREZ-MICHOACAN,1089.0,36 PUEBLA-VERACRUZ,0,5,0,1
1,136,55,36,M,767.0,55 5 DE MAYO-BOLIVAR,1002.0,136 ÁLVARO OBREGÓN-TONALÁ,0,30,0,1
2,115,29,38,M,733.0,29 REFORMA-BUCARELI,803.0,115 ROMA-DINAMARCA,0,24,0,1
3,115,29,43,M,733.0,29 REFORMA-BUCARELI,803.0,115 ROMA-DINAMARCA,0,24,0,1
4,35,136,37,M,1002.0,136 ÁLVARO OBREGÓN-TONALÁ,NaN,NaN,0,7,0,1


In [33]:
print(df_viajes_ageb['CVE_AGEB_retiro'].nunique(),df_viajes_ageb['CVE_AGEB_arribo'].nunique())

128 128


In [39]:
def periodo_hora(row):
    if row in [7,8,9]:
        return "7-10"
    elif row in[10,11,12]:
        return "10-13"
    elif row in [13,14,15]:
        return "13-16"
    elif row in [16,17,18]:
        return "16-19"
    elif row in [19,20,21]:
        return "19-22"
    elif row in [22,23,0]:
        return "22-1"
    else:
        return None

df_viajes_ageb["Hora_Retiro_bin"] = df_viajes_ageb["Hora_Retiro_round"].apply(periodo_hora)

df_viajes_ageb = df_viajes_ageb.dropna(subset=["Hora_Retiro_bin"],axis=0)

In [40]:
df_viajes_ageb_agg = df_viajes_ageb.groupby(['CVE_AGEB_arribo','CVE_AGEB_retiro','Hora_Retiro_bin' ]).agg({'Edad_Usuario':['count', 'mean'], 
                         'duracion_viaje_minutos':['mean','min','max'],
                                                                                       'Genero_Usuario_F':['sum'],
                                                                                       'Genero_Usuario_M':['sum'],
                                                                                      
                                                                                      }).reset_index(drop=False)
df_viajes_ageb_agg.columns = ['_'.join(col) for col in df_viajes_ageb_agg.columns]

In [42]:
df_viajes_ageb_agg.head(5)

,CVE_AGEB_arribo_,CVE_AGEB_retiro_,Hora_Retiro_bin_,Edad_Usuario_count,Edad_Usuario_mean,duracion_viaje_minutos_mean,duracion_viaje_minutos_min,duracion_viaje_minutos_max,Genero_Usuario_F_sum,Genero_Usuario_M_sum
0,11.0,11.0,10-13,282,37.982270,7.276596,1,56,45.0,237.0
1,11.0,11.0,13-16,327,37.721713,11.602446,1,58,56.0,271.0
2,11.0,11.0,16-19,355,36.591549,12.278873,1,59,67.0,288.0
3,11.0,11.0,19-22,295,34.325424,10.728814,1,51,83.0,212.0
4,11.0,11.0,22-1,83,32.578313,9.060241,1,44,21.0,62.0


Definimos una función para obtener el número de mujeres y de hombres:

In [44]:
def procentaje_homvres(row):
    return round(row['Genero_Usuario_M_sum']*10 / (row['Genero_Usuario_F_sum'] + row['Genero_Usuario_M_sum']),0)
    

df_viajes_ageb_agg['porcentage_hombres'] = df_viajes_ageb_agg.apply(procentaje_homvres, axis=1)

df_viajes_ageb_agg['porcentage_mujeres'] = 10 - df_viajes_ageb_agg['porcentage_hombres']

In [46]:
df_viajes_ageb_agg.head()

,CVE_AGEB_arribo_,CVE_AGEB_retiro_,Hora_Retiro_bin_,Edad_Usuario_count,Edad_Usuario_mean,duracion_viaje_minutos_mean,duracion_viaje_minutos_min,duracion_viaje_minutos_max,Genero_Usuario_F_sum,Genero_Usuario_M_sum,porcentage_hombres,porcentage_mujeres
0,11.0,11.0,10-13,282,37.982270,7.276596,1,56,45.0,237.0,8.0,2.0
1,11.0,11.0,13-16,327,37.721713,11.602446,1,58,56.0,271.0,8.0,2.0
2,11.0,11.0,16-19,355,36.591549,12.278873,1,59,67.0,288.0,8.0,2.0
3,11.0,11.0,19-22,295,34.325424,10.728814,1,51,83.0,212.0,7.0,3.0
4,11.0,11.0,22-1,83,32.578313,9.060241,1,44,21.0,62.0,7.0,3.0


In [45]:
df_viajes_ageb_agg.to_csv('../data/production_data/viajes_ecobici.csv',index=False)